In [1]:
import warnings
warnings.filterwarnings('ignore')
from cliotools.pcaskysub import build_raw_stack, build_skysubbed_stack, mask, highpassfilter,\
    findbadpix, badpixfix, plot, pd, np
import IPython

# Make a stack of the raw images in the dataset: 
path = "Data/HD_142415/"

k = pd.read_csv(path+'ABLocations.txt', 
                     sep = ',', # Commas separate items in the list
                     comment = '#', # Ignore commented rows
                     names=['filename', 'xca','yca', 'xcb', 'ycb'] # specify column names
                    )
stack0, stack1, xca0, yca0, xcb0, ycb0, xca1, yca1, xcb1, ycb1 = build_raw_stack(k)

# High pass filter (unsharp mask) to remove spatial variation:
print('Highpassfilter Nod 0:')
hpf0 = highpassfilter(stack0) 
print('Highpassfilter Nod 1:')
hpf1 = highpassfilter(stack1)
# mask stars in each nod:
print('Masking stars')
masked0 = hpf0.copy()
for i in range(stack0.shape[0]):
    masked0[i,:,:] = mask(hpf0[i,:,:],[xca0[i],xcb0[i]],[yca0[i],ycb0[i]], radius = 80)
masked1 = hpf1.copy()
for i in range(stack1.shape[0]):
    masked1[i,:,:] = mask(hpf1[i,:,:],[xca1[i],xcb1[i]],[yca1[i],ycb1[i]], radius = 80)

print('Taking time-domain std dev')
# Take standard deviation in time domain:
std0 = np.nanstd(masked0, axis=0)
std1 = np.nanstd(masked1, axis=0)

Stacking reference images for  ...
I found  10  images for Nod 0, and  10 images for Nod 1
Highpassfilter Nod 0:
100% (10 of 10): |####################|  Done...
Highpassfilter Nod 1:
100% (10 of 10): |####################|  Done...
Masking stars
Taking time-domain std dev


In [2]:
print('Find a chunk with no bad pixels:')
# Find a chunk of image with no visible bad pixels:
%matplotlib notebook
plot(std1)

Find a chunk with no bad pixels:


<IPython.core.display.Javascript object>

In [3]:
print('Find a chunk with no bad pixels:')
# Find a chunk of image with no visible bad pixels:
%matplotlib notebook
plot(std0)

Find a chunk with no bad pixels:


In [4]:
xmin, xmax = 185,215
ymin, ymax = 455,495
chunk0 = std0[ymin:ymax,xmin:xmax]
xmin, xmax = 315,352
ymin, ymax = 218,250
chunk1 = std1[ymin:ymax,xmin:xmax]

# Make a bad pix map for each nod:
print('Finding badpix')
n = 3
badpix0, badpixmap0 = findbadpix(std0, chunk0, n=n)
badpix1, badpixmap1 = findbadpix(std1, chunk1, n=n)

print('Opening skysubs')
# Make a new stack of the skysubbtracted images to fix:
images0,images1 = build_skysubbed_stack(k)
# Use the badpix index list to fix pixels in opposite nod:
print('Nod1:')
imfix1 = []
imfix0 = []
for i in range(10):
    imfix1.append(badpixfix(images1[i],badpix0,xca1[i], yca1[i], xcb1[i], ycb1[i]))
print('Nod0:')
for i in range(10):
    imfix0.append(badpixfix(images0[i],badpix1, xca0[i], yca0[i], xcb0[i], ycb0[i]))

Finding badpix
Opening skysubs
Nod1:
Nod0:


In [5]:
%matplotlib notebook
plot(imfix1[1])

In [6]:
# Spot healing: pixels that need to be fixed near stars that weren't caught by automatic pipeline:
shx = [605,605,573,556,648,683,691,690,690,617,617,575,706,717,742,766,818,807,807,606,606,807,807,606,778,778,\
      779,779,570,570,617,617]
shy = [152,150,118,149,158,125,125,108,107,37,36,68,4,2,4,67,108,318,317,152,150,137,136,149,116,115,\
      116,115,28,31,35,38]

imfix1_2 = []
for i in range(len(imfix1)):
    imfix1_2.append(badpixfix(imfix1[i],[shy,shx], xca1[i],yca1[i],xcb1[i],ycb1[i]))
%matplotlib notebook
plot(imfix1_2[1])

In [7]:
# Check your work:
%matplotlib notebook
plot(imfix0[0])

In [8]:
# Spot healing: pixels that need to be fixed near stars that weren't caught by automatic pipeline:
shx = [205,205,206,292]
shy = [320,319,320,299]

    
imfix0_2 = []
for i in range(len(imfix0)):
    imfix0_2.append(badpixfix(imfix0[i],[shy,shx], xca0[i],yca0[i],xcb0[i],ycb0[i]))
%matplotlib notebook
plot(imfix1_2[1])

In [12]:
# Write out fixed images:
count0, count1 = 0,0
from astropy.io import fits
import time
import os
for i in range(len(k)):
    sp = k['filename'][i].split(',')
    filename = sp[0]
    
    imhdr = fits.getheader(filename)
    imhdr['COMMENT'] = '    deep bad pixel corrected on '\
                           +time.strftime("%m/%d/%Y")+ ' By Logan A. Pearce'
    
    newname = filename.split('.')[0] + '_ssbp.fit'
    if imhdr['BEAM'] == 0:
        print(newname, imhdr['BEAM'], count0)
        fits.writeto(newname, imfix0[count0], imhdr, overwrite=True)
        count0 += 1
    if imhdr['BEAM'] == 1:
        print(newname, imhdr['BEAM'], count1)
        fits.writeto(newname, imfix1[count1], imhdr, overwrite=True)
        count1 += 1
        
    # Compare the original and new FITS files
    original_data = fits.getdata(filename)
    new_data = fits.getdata(newname)

    # Verify successful writing
    if os.path.exists(newname): 
        print(f"Successfully wrote {newname}") 
        if np.array_equal(original_data, new_data):
            print(f"{filename[-20:]} and the new file are exactly equal!!!!!!!!")
        else:
            print(f"{filename[-20:]} and {newname} are NOT equal.")
            # Find and print where they differ
            #diff_indices = np.where(original_data != new_data)
            #for index in zip(*diff_indices):
                #print(f"Difference at index {index}: original={original_data[index]}, new={new_data[index]}")
    else: 
        print(f"Failed to write {newname}")


/Users/bioversedeveloper/new_clio/badpixfix/cliotools/Data/HD_142415/HD142415Ks_00001_ssbp.fit 0 0
Successfully wrote /Users/bioversedeveloper/new_clio/badpixfix/cliotools/Data/HD_142415/HD142415Ks_00001_ssbp.fit
HD142415Ks_00001.fit and /Users/bioversedeveloper/new_clio/badpixfix/cliotools/Data/HD_142415/HD142415Ks_00001_ssbp.fit are NOT equal.
Difference at index (np.int64(0), np.int64(0)): original=144437, new=119159.0
Difference at index (np.int64(0), np.int64(1)): original=104875, new=117047.0
Difference at index (np.int64(0), np.int64(2)): original=117047, new=130109.0
Difference at index (np.int64(0), np.int64(3)): original=119159, new=124960.0
Difference at index (np.int64(0), np.int64(4)): original=116401, new=138241.0
Difference at index (np.int64(0), np.int64(5)): original=141923, new=132873.0
Difference at index (np.int64(0), np.int64(6)): original=132873, new=137751.0
Difference at index (np.int64(0), np.int64(7)): original=141059, new=135570.5
Difference at index (np.int6

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [10]:
# Finally, write out the list of sky subbed, bad pixel fixed, cleaned images:
g = k.copy()
for i in range(len(k)):
    newname = (k['filename'][i]).split('_')[0]+'_'+(k['filename'][i]).split('_')[1]+'_ssbp.fit'
    g['filename'][i] = newname
g.to_csv(path+'CleanList', index=False)

In [11]:
m = pd.read_csv(path+'CleanList')
m

,filename,xca,yca,xcb,ycb
0,/Users/bioversedeveloper/new_clio/badpixfix/cl...,176.359617,351.000000,177.000000,352.00000
1,/Users/bioversedeveloper/new_clio/badpixfix/cl...,176.359617,351.000000,177.000000,352.00000
2,/Users/bioversedeveloper/new_clio/badpixfix/cl...,176.359617,351.000000,177.000000,352.00000
3,/Users/bioversedeveloper/new_clio/badpixfix/cl...,176.359617,351.000000,177.000000,352.00000
4,/Users/bioversedeveloper/new_clio/badpixfix/cl...,176.359617,351.000000,177.000000,352.00000
5,/Users/bioversedeveloper/new_clio/badpixfix/cl...,427.325668,353.901055,417.000000,535.00000
6,/Users/bioversedeveloper/new_clio/badpixfix/cl...,427.325668,353.901055,417.000000,535.00000
7,/Users/bioversedeveloper/new_clio/badpixfix/cl...,427.325668,353.901055,417.000000,535.00000
8,/Users/bioversedeveloper/new_clio/badpixfix/cl...,427.325668,353.901055,417.000000,535.00000
9,/Users/bioversedeveloper/new_clio/badpixfix/cl...,427.215197,353.801048,384.902303,377.42775
